<h1>Queries</h1>

In [0]:
# load data
tweets = spark.read.json('/mnt/ddscoursedatabricksstg/ddscoursedatabricksdata/coronavirus-tweets/')

# imports
from pyspark.sql.window import Window as W
import pyspark.sql.functions as F
import re

# remove punc + lowercase text
@udf
def clean(s):
  res = re.sub(r'[^\w\s]',' ',s)
  final_res = re.sub(r'[\n\r\t]',' ',res)
  return final_res.lower()

tweets = tweets.withColumn('cleaned_txt', clean(F.col('text'))).drop('text').withColumnRenamed('cleaned_txt', 'text')
tweets = tweets.drop_duplicates(subset=['id_str'])

# variables and views:
tweets.createOrReplaceTempView("tweets") # for sql queries
date_format = 'EEE MMM dd HH:mm:ss +SSSS yyyy' # for converting timestamp into datetime

<h3>Q1: How many tweets contains the word mask (text field), per day? Provide a histogram.</h3>

<h4>SQL</h4>

In [0]:
%sql
SELECT date, COUNT(*) as count
FROM
  (SELECT *, DATE(to_timestamp(created_at, 'EEE MMM dd HH:mm:ss +SSSS yyyy')) AS date
  FROM tweets 
  WHERE text LIKE '% mask %' OR text LIKE 'mask %' OR text LIKE '% mask') 
GROUP BY date
ORDER BY date

date,count
2020-01-27,694
2020-01-28,1060
2020-01-29,2673
2020-01-30,6362
2020-01-31,2227
2020-02-01,844
2020-02-02,1050
2020-02-03,2689
2020-02-04,2912
2020-02-05,4541


<h4>DataFrame</h4>

In [0]:
df = tweets
df = df.where(F.col('text').like("% mask %") | F.col('text').like("mask %") | F.col('text').like("% mask")) 
df = df.withColumn('date', F.to_timestamp(df.created_at, date_format)).withColumn('day', F.to_date('date'))
df = df.groupBy(F.col('day')).count().sort(F.asc("day"))
display(df)

day,count
2020-01-27,694
2020-01-28,1060
2020-01-29,2673
2020-01-30,6362
2020-01-31,2227
2020-02-01,844
2020-02-02,1050
2020-02-03,2689
2020-02-04,2912
2020-02-05,4541


<h4>Histogram</h4>

In [0]:
# create dates range dataframe
l = df.select([F.min("day")]).collect()[0][0]
u = df.select([F.max("day")]).collect()[0][0]

df1 = spark.createDataFrame([(l, u)], ('C1', 'C2'))
df2 = df1.select(F.sequence('C1', 'C2').alias('r'))
df3 = df2.withColumn('day', F.explode(F.col('r'))).drop('r')

# join & display
hist = df.join(df3, 'day', 'rightouter').fillna(0).sort('day')
display(hist)

day,count
2020-01-27,694
2020-01-28,1060
2020-01-29,2673
2020-01-30,6362
2020-01-31,2227
2020-02-01,844
2020-02-02,1050
2020-02-03,2689
2020-02-04,2912
2020-02-05,4541


<h4>Bonus</h4>

For each date we wanted to examine the ratio between tweets containing the word 'mask' and total tweets count:
<br>The first histogram shows for each day both counts - for mask tweets and for all tweets.
<br>The second histogram shows for each day the ratio between these parameters.

In [0]:
# foreach day, count tweets with mask (same as q1):
df1 = tweets
df1 = df1.where(F.col('text').like("% mask %") | F.col('text').like("mask %") | F.col('text').like("% mask")) 
df1 = df1.withColumn('date', F.to_timestamp(df1.created_at, date_format)).withColumn('day', F.to_date('date'))
df1 = df1.groupBy(F.col('day')).count().sort(F.asc("day")).withColumnRenamed('count', 'mask_count')

# foreach day, count all tweets:
df2 = tweets
df2 = df2.withColumn('date', F.to_timestamp(df2.created_at, date_format)).withColumn('day', F.to_date('date'))
df2 = df2.groupBy(F.col('day')).count().sort(F.asc("day")).withColumnRenamed('count', 'total_count')

res = df1.join(df2, 'day', 'inner')

# visualization with gaps-filling - same as q1 histogram:
l = res.select([F.min("day")]).collect()[0][0]
u = res.select([F.max("day")]).collect()[0][0]

df1 = spark.createDataFrame([(l, u)], ('C1', 'C2'))
df2 = df1.select(F.sequence('C1', 'C2').alias('r'))
df3 = df2.withColumn('day', F.explode(F.col('r'))).drop('r')

# join & display
hist = res.join(df3, 'day', 'rightouter').fillna(0).sort('day')
display(hist)

day,mask_count,total_count
2020-01-27,694,50221
2020-01-28,1060,52750
2020-01-29,2673,302545
2020-01-30,6362,121766
2020-01-31,2227,107557
2020-02-01,844,60116
2020-02-02,1050,100281
2020-02-03,2689,413303
2020-02-04,2912,524124
2020-02-05,4541,432147


In [0]:
# compute ratio between mask tweets and total tweets count
@udf
def r(x, y):
  if y > 0:
    return x / y
  else:
    return 0

hist = hist.withColumn('ratio', r(F.col('mask_count'), F.col('total_count')))
display(hist)

day,mask_count,total_count,ratio
2020-01-27,694,50221,0.013818920371955954
2020-01-28,1060,52750,0.02009478672985782
2020-01-29,2673,302545,0.008835049331504405
2020-01-30,6362,121766,0.052247753888606016
2020-01-31,2227,107557,0.02070530044534526
2020-02-01,844,60116,0.014039523587730389
2020-02-02,1050,100281,0.010470577676728394
2020-02-03,2689,413303,0.006506122626741156
2020-02-04,2912,524124,0.00555593714464516
2020-02-05,4541,432147,0.010507998435717475


<h3>Q2: How many tweets there are per hour, for each source? Provide a histogram for the top 5 source.</h3>

<h4>SQL</h4>

In [0]:
%sql
SELECT T.source, HOUR(to_timestamp(created_at, 'EEE MMM dd HH:mm:ss +SSSS yyyy')) AS hour, COUNT(id_str) AS count
FROM tweets T
GROUP BY T.source, hour
ORDER BY hour ASC

source,hour,count
Twithaca,0,1
auto is the only way it can be,0,2
Yamaha R1M,0,2
Hootsuite Inc.,0,113
PortalPortuario,0,1
EcoInternet3,0,16
Twitter BQuarks G #2,0,1
drobapi,0,1
Sprinklr,0,1
Tabtter Free,0,2


<h4>DataFrame</h4>

In [0]:
df = tweets
df = df.withColumn('date', F.to_timestamp(df.created_at, date_format)).withColumn('hour', F.hour('date'))
df = df.groupBy('source', 'hour').count().sort(F.asc("hour"))
display(df)

source,hour,count
Periscope,0,9
Tim Wilson,0,1
Tweetpodernfam,0,1
test33123112,0,1
CybazeSocial,0,1
Tweetbot for iΟS,0,28
tmr - Alternative,0,1
OAuth Gateway App,0,1
ncov_retweet_scientific,0,1
twinTweetbolt,0,1


<h4>Histogram</h4>

In [0]:
src = tweets.groupBy('source').count().sort(F.desc("count")).limit(5).drop('count')
df = df.join(src, 'source', 'inner')

df1 = spark.createDataFrame([(0, 23)], ('C1', 'C2'))
df2 = df1.select(F.sequence('C1', 'C2').alias('r'))
df3 = df2.withColumn('hour', F.explode(F.col('r'))).drop('r')

# filter source name
@udf
def name(s):
  tmp1 = s.split('>')[-2]
  res = tmp1.split('<')[0]
  return res

# join & display
hist = df.join(df3, 'hour', 'rightouter').fillna(0)
hist = hist.withColumn('source_name', name(F.col('source'))).drop('source').sort('hour')
display(hist)

hour,count,source_name
0,551,TweetDeck
0,1326,Twitter for iPad
0,12536,Twitter for iPhone
0,15940,Twitter for Android
0,9127,Twitter Web App
1,1971,Twitter for iPad
1,12204,Twitter Web App
1,18420,Twitter for iPhone
1,22641,Twitter for Android
1,716,TweetDeck


<h4>Bonus</h4>

To examine trends of using different devices as a function of the hour, we display the query result as a line plot:

In [0]:
display(hist)

hour,count,source_name
0,551,TweetDeck
0,12536,Twitter for iPhone
0,15940,Twitter for Android
0,9127,Twitter Web App
0,1326,Twitter for iPad
1,716,TweetDeck
1,12204,Twitter Web App
1,18420,Twitter for iPhone
1,22641,Twitter for Android
1,1971,Twitter for iPad


<h3>Q3: Which hashtag has the most tweets that contains the word mask, per 4 hours? Provide a histogram.</h3>

<h4>SQL</h4>

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW DF AS
SELECT hashtag, lower, upper, COUNT(*) AS count
FROM
(
    SELECT DISTINCT T.id, HASH.hashtag, 
                 CEILING((HOUR(to_timestamp(created_at, 'EEE MMM dd HH:mm:ss +SSSS yyyy'))+1)/4)*4 - 4 AS lower, 
                 CEILING((HOUR(to_timestamp(created_at, 'EEE MMM dd HH:mm:ss +SSSS yyyy'))+1)/4)*4 AS upper
    FROM
    (
      SELECT id, explode(entities.hashtags.text) AS hashtag FROM tweets
        UNION
      SELECT id, explode(retweeted_status.entities.hashtags.text) AS hashtag FROM tweets
        UNION
      SELECT id, explode(quoted_status.entities.hashtags.text) AS hashtag FROM tweets
    ) HASH, tweets T
    WHERE HASH.id == T.id AND (text LIKE '% mask %' OR text LIKE 'mask %' OR text LIKE '% mask')
)
GROUP BY hashtag, lower, upper
ORDER BY lower ASC, count DESC

In [0]:
%sql
SELECT DF.lower, DF.upper, DF.hashtag, DF.count
FROM 
  (SELECT MAX(count) AS max_count, lower, upper
  FROM DF
  GROUP BY Lower, upper) MAXIMUM, DF
WHERE MAXIMUM.max_count == DF.count AND MAXIMUM.lower == DF.lower AND  MAXIMUM.upper == DF.upper 
ORDER BY DF.lower

lower,upper,hashtag,count
0,4,coronavirus,864
4,8,coronavirus,1078
8,12,coronavirus,1222
12,16,coronavirus,1630
16,20,coronavirus,1111
20,24,coronavirus,589


<h4>DataFrame</h4>

In [0]:
df = tweets

df = df.where(F.col('text').like("% mask %") | F.col('text').like("mask %") | F.col('text').like("% mask")) 

df1 = df.withColumn("hashtag", F.explode(df.entities.hashtags.text)) 
df2 = df.withColumn("hashtag", F.explode(df.retweeted_status.entities.hashtags.text))
df3 = df.withColumn("hashtag", F.explode(df.quoted_status.entities.hashtags.text))
df = df1.union(df2).union(df3).dropDuplicates()

df = df.withColumn('date', F.to_timestamp(df.created_at, date_format)).withColumn('hour', F.hour('date'))
df = df.groupBy(F.col('hashtag'), F.window(F.col('date'), '4 hour')).count()

df = df.withColumn("start", F.explode(F.array(F.col('window')).getItem('start'))).withColumn('start_h', F.hour('start'))
df = df.withColumn("end", F.explode(F.array(F.col('window')).getItem('end'))).withColumn('end_h', F.hour('end'))

df = df.select('hashtag', 'count', 'start_h', 'end_h')
df = df.groupBy(F.col('hashtag'), F.col('start_h'), F.col('end_h')).sum('count').sort(F.desc("sum(count)"))

res = df.groupBy("start_h", "end_h").agg(F.max(F.struct(F.col("sum(count)"), F.col("hashtag"))).alias("max"))\
.select(F.col("start_h"), F.col("end_h"), F.col("max.hashtag"), F.col("max.sum(count)")).sort('start_h')
display(res)

start_h,end_h,hashtag,sum(count)
0,4,coronavirus,864
4,8,coronavirus,1078
8,12,coronavirus,1222
12,16,coronavirus,1630
16,20,coronavirus,1111
20,0,coronavirus,589


In [0]:
display(res)

start_h,end_h,hashtag,sum(count)
0,4,coronavirus,864
4,8,coronavirus,1078
8,12,coronavirus,1222
12,16,coronavirus,1630
16,20,coronavirus,1111
20,0,coronavirus,589


<h4>Bonus</h4>

'coronavirus' is the most popular hashtag in all intervals, but we want to show it in comparison to other hashtags.
<br>Thus, we have created a pie chart for most intervals to demonstrate the hashtags distribution.

In [0]:
display(df)

hashtag,start_h,end_h,sum(count)
coronavirus,12,16,1630
coronavirus,8,12,1222
coronavirus,16,20,1111
coronavirus,4,8,1078
coronavirus,0,4,864
coronavirus,20,0,589
CoronaVirus,8,12,468
China,8,12,446
Florence,8,12,417
CoronaOutbreak,4,8,403


<h3>Q4: How many users, per lang, tweeted above the median tweet count, per day? Provide a histogram.</h3>

<h4>SQL</h4>

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW DF AS
(SELECT lang, day, user_id, COUNT(*) AS usr_cnt 
FROM
  (SELECT *, explode(array(user.id_str)) AS user_id, DATE(to_timestamp(created_at, 'EEE MMM dd HH:mm:ss +SSSS yyyy')) AS day
  FROM tweets) T
GROUP BY lang, day, user_id)

In [0]:
%sql
SELECT DF.day, DF.lang, COUNT(DISTINCT DF.user_id) AS user_count
FROM DF, 
  (
  SELECT lang, day, percentile_approx(DF.usr_cnt, 0.5) AS median
  FROM DF
  GROUP BY lang, day
  ) MED
WHERE MED.lang == DF.lang AND MED.day == DF.day AND DF.usr_cnt > MED.median
GROUP BY DF.day, DF.lang

day,lang,user_count
2020-02-02,fi,1
2020-01-31,lt,1
2020-01-29,en,20142
2020-02-04,en,42357
2020-01-29,ko,61
2020-01-30,ja,90
2020-03-02,es,315
2020-02-06,fi,1
2020-02-05,in,532
2020-02-06,te,1


<h4>DataFrame</h4>

In [0]:
df = tweets

df = df.withColumn("user_id", F.explode(F.array(df.user).getItem("id_str")))
df = df.withColumn('date', F.to_timestamp(df.created_at, date_format)).withColumn('day', F.to_date('date'))
df = df.groupBy('day', 'lang', 'user_id').count().withColumnRenamed('count', 'usr_cnt')

median_ex = F.expr('percentile_approx(usr_cnt, 0.5)')
df_gb = df.groupBy('day', 'lang').agg(median_ex.alias('med_val'))

res = df.join(df_gb, ['day', 'lang'], 'inner').where(df.usr_cnt > df_gb.med_val)
res = res.groupBy('day', 'lang').count()

display(res)

day,lang,count
2020-02-04,th,3260
2020-01-29,en,20142
2020-01-29,in,1156
2020-01-29,pa,1
2020-01-29,te,10
2020-01-31,it,1349
2020-02-04,pt,306
2020-02-04,sr,1
2020-02-06,th,159
2020-01-29,fr,1233


<h4>Histogram</h4>

In [0]:
# create dates range dataframe
l = res.select([F.min("day")]).collect()[0][0]
u = res.select([F.max("day")]).collect()[0][0]

df1 = spark.createDataFrame([(l, u)], ('C1', 'C2'))
df2 = df1.select(F.sequence('C1', 'C2').alias('r'))
df3 = df2.withColumn('day', F.explode(F.col('r'))).drop('r')
df4 = tweets.select('lang').dropDuplicates().crossJoin(df3)

# join & display
hist = res.join(df4, ['day', 'lang'], 'rightouter').fillna(0).sort('day')
display(hist.sort(F.desc('count')).sort('day'))

day,lang,count
2020-01-27,cy,0
2020-01-27,fr,358
2020-01-27,ne,0
2020-01-27,ht,0
2020-01-27,sr,0
2020-01-27,cs,0
2020-01-27,uk,0
2020-01-27,my,0
2020-01-27,ml,0
2020-01-27,tl,6


<h4>Bonus</h4>

English is the most popular language by far, so it is reasonable that the users count of the English language in the histogram above will be the highest.
<br>Thus we have decided to visualize the trends of the top 5 popular languages, beside English, for each day in the data.

In [0]:
top5 = tweets.where((F.col('lang') != 'en') & (F.col('lang') != 'und'))
top5 = top5.groupBy('lang').count().sort(F.desc('count')).limit(5).select('lang')
hist2 = top5.join(hist, 'lang', 'inner').sort(F.desc('count')).sort('day')
display(hist2)

lang,day,count
th,2020-01-27,55
es,2020-01-27,367
in,2020-01-27,73
it,2020-01-27,52
fr,2020-01-27,358
th,2020-01-28,87
in,2020-01-28,134
es,2020-01-28,826
fr,2020-01-28,66
it,2020-01-28,17
